In [ ]:
from functools import reduce
import pandas as pd
from db_queries import get_location_metadata

import warnings
warnings.simplefilter('ignore')

DATE_RANGES = [(pd.to_datetime('2020-05-18'), pd.to_datetime('2020-05-25')),
               (pd.to_datetime('2020-05-25'), pd.to_datetime('2020-06-01'))]


In [ ]:
loc_df = get_location_metadata(location_set_id=111, location_set_version_id=698)
loc_df = loc_df.loc[loc_df['path_to_top_parent'].str.startswith('102,')]

df = pd.read_csv('/ihme/covid-19/deaths/dev/2020_05_25_cfrmod/model_results.csv')
df['date'] = pd.to_datetime(df['date'])
df['deaths'] = df[[f'draw_{d}' for d in range(1000)]].mean(axis=1)
df = df[['location_id', 'date', 'deaths']]

start_date = df['date'] == df.groupby('location_id')['date'].transform(min)
df = df.sort_values(['location_id', 'date'])
df['daily_deaths'] = np.nan
df['daily_deaths'][1:] = np.diff(df['deaths'])
df.loc[start_date, 'daily_deaths'] = df['deaths']

df = loc_df[['location_id', 'location_name', 'parent_id']].merge(df)
df.loc[df['parent_id'] == 570, 'location_id'] = 570
df.loc[df['parent_id'] == 570, 'location_name'] = 'Washington'
df = df.groupby(['location_id', 'location_name', 'date'], as_index=False)['daily_deaths'].sum()


In [ ]:
def get_pct_change(df, date_range):
    df = df.copy()
    df = df.loc[df['date'].isin(date_range)]
    start = date_range[0].strftime('%m_%d')
    end = date_range[1].strftime('%m_%d')
    df.loc[df['date'] == date_range[0], 'range_point'] = start
    df.loc[df['date'] == date_range[1], 'range_point'] = end
    df = pd.pivot_table(df, index=['location_id', 'location_name'], 
                        columns='range_point', values='daily_deaths').reset_index()
    colname = f'pct_change_{start}_to_{end}'
    df[colname] = ((df[end] - df[start]) / df[start]) * 100

    return df[['location_id', 'location_name', start, end, colname]]


In [ ]:
dfs = [get_pct_change(df, date_range) for date_range in DATE_RANGES]

pct_change_df = reduce(lambda x, y: pd.merge(x, y, how='outer'), dfs)
pct_change_df.to_csv('/ihme/homes/rmbarber/covid-19/change_0518_0525_0601.csv', index=False)
